In [5]:
pip install openmeteo-requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.1/167.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.2/684.2 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.1/394.1 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.7 MB/s eta 0:00:00


In [6]:
pip install requests-cache retry-requests numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 6.0 MB/s eta 0:00:00


In [9]:
import requests_cache
from retry_requests import retry
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [13]:

# Setup the Open-Meteo API client with cache and retries
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

In [11]:
# Your 50 cities
CITIES = [
    {'name': 'Mumbai', 'lat': 19.0760, 'lon': 72.8777},
    {'name': 'Delhi', 'lat': 28.7041, 'lon': 77.1025},
    {'name': 'Bangalore', 'lat': 12.9716, 'lon': 77.5946},
    {'name': 'Hyderabad', 'lat': 17.3850, 'lon': 78.4867},
    {'name': 'Ahmedabad', 'lat': 23.0225, 'lon': 72.5714},
    {'name': 'Chennai', 'lat': 13.0827, 'lon': 80.2707},
    {'name': 'Kolkata', 'lat': 22.5726, 'lon': 88.3639},
    {'name': 'Surat', 'lat': 21.1702, 'lon': 72.8311},
    {'name': 'Pune', 'lat': 18.5204, 'lon': 73.8567},
    {'name': 'Jaipur', 'lat': 26.9124, 'lon': 75.7873},
    {'name': 'Lucknow', 'lat': 26.8467, 'lon': 80.9462},
    {'name': 'Kanpur', 'lat': 26.4499, 'lon': 80.3319},
    {'name': 'Nagpur', 'lat': 21.1458, 'lon': 79.0882},
    {'name': 'Indore', 'lat': 22.7196, 'lon': 75.8577},
    {'name': 'Thane', 'lat': 19.2183, 'lon': 72.9781},
    {'name': 'Bhopal', 'lat': 23.2599, 'lon': 77.4126},
    {'name': 'Visakhapatnam', 'lat': 17.6868, 'lon': 83.2185},
    {'name': 'Pimpri-Chinchwad', 'lat': 18.6298, 'lon': 73.7997},
    {'name': 'Patna', 'lat': 25.5941, 'lon': 85.1376},
    {'name': 'Vadodara', 'lat': 22.3072, 'lon': 73.1812},
    {'name': 'Ghaziabad', 'lat': 28.6692, 'lon': 77.4538},
    {'name': 'Ludhiana', 'lat': 30.9010, 'lon': 75.8573},
    {'name': 'Agra', 'lat': 27.1767, 'lon': 78.0081},
    {'name': 'Nashik', 'lat': 19.9975, 'lon': 73.7898},
    {'name': 'Faridabad', 'lat': 28.4089, 'lon': 77.3178},
    {'name': 'Meerut', 'lat': 28.9845, 'lon': 77.7064},
    {'name': 'Rajkot', 'lat': 22.3039, 'lon': 70.8022},
    {'name': 'Kalyan-Dombivli', 'lat': 19.2403, 'lon': 73.1305},
    {'name': 'Vasai-Virar', 'lat': 19.3919, 'lon': 72.8397},
    {'name': 'Varanasi', 'lat': 25.3176, 'lon': 82.9739},
    {'name': 'Coimbatore', 'lat': 11.0168, 'lon': 76.9558},
    {'name': 'Madurai', 'lat': 9.9252, 'lon': 78.1198},
    {'name': 'Amritsar', 'lat': 31.6340, 'lon': 74.8723},
    {'name': 'Jamshedpur', 'lat': 22.8046, 'lon': 86.2029},
    {'name': 'Allahabad', 'lat': 25.4358, 'lon': 81.8463},
    {'name': 'Ranchi', 'lat': 23.3441, 'lon': 85.3096},
    {'name': 'Haora', 'lat': 22.5896, 'lon': 88.3103},
    {'name': 'Gwalior', 'lat': 26.2183, 'lon': 78.1828},
    {'name': 'Vijayawada', 'lat': 16.5062, 'lon': 80.6480},
    {'name': 'Chandigarh', 'lat': 30.7333, 'lon': 76.7794},
    {'name': 'Guwahati', 'lat': 26.1445, 'lon': 91.7362},
    {'name': 'Solapur', 'lat': 17.6599, 'lon': 75.9064},
    {'name': 'Hubli-Dharwad', 'lat': 15.3647, 'lon': 75.1240},
    {'name': 'Bareilly', 'lat': 28.3670, 'lon': 79.4304},
    {'name': 'Moradabad', 'lat': 28.8252, 'lon': 78.6732},
    {'name': 'Mysuru', 'lat': 12.2958, 'lon': 76.6394},
    {'name': 'Tiruchirappalli', 'lat': 10.7905, 'lon': 78.7047},
    {'name': 'Jalandhar', 'lat': 31.3260, 'lon': 75.5762},
    {'name': 'Bhubaneswar', 'lat': 20.2961, 'lon': 85.8245},
    {'name': 'Salem', 'lat': 11.6643, 'lon': 78.1460},
]


In [14]:
def get_season(month):
    if month in [12, 1, 2]: return "Winter"
    elif month in [3, 4, 5]: return "Summer"
    elif month in [6, 7, 8, 9]: return "Monsoon"
    else: return "Post-monsoon"

def assign_quarter(hour):
    if 0 <= hour < 6: return "Q1"
    elif 6 <= hour < 12: return "Q2"
    elif 12 <= hour < 18: return "Q3"
    else: return "Q4"

# Fetch data for all cities
all_results = []

for idx, city in enumerate(CITIES):
    print(f"[{idx+1}/{len(CITIES)}] Fetching for {city['name']}")
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": city['lat'],
        "longitude": city['lon'],
        "start_date": "2023-01-01",
        "end_date": "2025-11-24",
        "hourly": [
            "temperature_2m",
            "relative_humidity_2m",
            "apparent_temperature",
            "surface_pressure",
            "wind_speed_10m",
            "pressure_msl",
            "dew_point_2m",
            "cloud_cover",
            "et0_fao_evapotranspiration",
            "vapour_pressure_deficit",
            "precipitation"
        ],
    }
    max_wait_retries = 20
    wait_retries = 0
    while True:
        try:
            responses = openmeteo.weather_api(url, params=params)
            response = responses[0]
            # --- Extract and process hourly data ---
            hourly = response.Hourly()
            hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
            hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
            hourly_apparent_temperature = hourly.Variables(2).ValuesAsNumpy()
            hourly_surface_pressure = hourly.Variables(3).ValuesAsNumpy()
            hourly_wind_speed_10m = hourly.Variables(4).ValuesAsNumpy()
            hourly_pressure_msl = hourly.Variables(5).ValuesAsNumpy()
            hourly_dew_point_2m = hourly.Variables(6).ValuesAsNumpy()
            hourly_cloud_cover = hourly.Variables(7).ValuesAsNumpy()
            hourly_et0_fao_evapotranspiration = hourly.Variables(8).ValuesAsNumpy()
            hourly_vapour_pressure_deficit = hourly.Variables(9).ValuesAsNumpy()
            hourly_precipitation = hourly.Variables(10).ValuesAsNumpy()

            # Build DataFrame
            hourly_data = {
                "datetime": pd.date_range(
                    start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                    end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                    freq=pd.Timedelta(seconds=hourly.Interval()),
                    inclusive="left"
                ),
                "temperature_2m": hourly_temperature_2m,
                "relative_humidity_2m": hourly_relative_humidity_2m,
                "apparent_temperature": hourly_apparent_temperature,
                "surface_pressure": hourly_surface_pressure,
                "wind_speed_10m": hourly_wind_speed_10m,
                "pressure_msl": hourly_pressure_msl,
                "dew_point_2m": hourly_dew_point_2m,
                "cloud_cover": hourly_cloud_cover,
                "et0_fao_evapotranspiration": hourly_et0_fao_evapotranspiration,
                "vapour_pressure_deficit": hourly_vapour_pressure_deficit,
                "precipitation": hourly_precipitation,
            }
            hourly_df = pd.DataFrame(hourly_data)
            hourly_df['date'] = hourly_df['datetime'].dt.date
            hourly_df['hour'] = hourly_df['datetime'].dt.hour
            hourly_df['month'] = hourly_df['datetime'].dt.month
            hourly_df['interval'] = hourly_df['hour'].apply(assign_quarter)
            hourly_df['season'] = hourly_df['month'].apply(get_season)
            hourly_df['city'] = city['name']
            hourly_df['lat'] = city['lat']
            hourly_df['lon'] = city['lon']

            # Aggregate Q1–Q4 per day
            agg_dict = {
                'temperature_2m': 'mean',
                'relative_humidity_2m': 'mean',
                'apparent_temperature': 'mean',
                'surface_pressure': 'mean',
                'wind_speed_10m': 'mean',
                'pressure_msl': 'mean',
                'dew_point_2m': 'mean',
                'cloud_cover': 'mean',
                'et0_fao_evapotranspiration': 'sum',
                'vapour_pressure_deficit': 'mean',
                'precipitation': 'sum',
                'month': 'first',
                'season': 'first',
                'city': 'first',
                'lat': 'first',
                'lon': 'first',
            }
            daily_q = hourly_df.groupby(['date', 'interval']).agg(agg_dict).reset_index()
            all_results.append(daily_q)
            break  # Success, move to next city
        except Exception as e:
            # Look for API minutely limit error
            if hasattr(e, 'args') and any(
                ("Minutely API request limit exceeded" in str(arg)) for arg in e.args
            ):
                wait_retries += 1
                wait_time = min(60, 10 * wait_retries)
                print(f"Rate limited for {city['name']}! Waiting {wait_time}s (retry {wait_retries}/{max_wait_retries})...")
                time.sleep(wait_time)
                if wait_retries > max_wait_retries:
                    print(f"Giving up on {city['name']} after {max_wait_retries} retries.")
                    break
                continue  # retry this city
            else:
                print(f"Permanent error for {city['name']}: {e}")
                break  # skip city on permanent error
    time.sleep(1)

# Combine and save to Excel
if all_results:
    final_df = pd.concat(all_results, ignore_index=True)
    final_df = final_df[[
        'city', 'date', 'month', 'season', 'interval', 'lat', 'lon',
        'temperature_2m', 'apparent_temperature', 'dew_point_2m',
        'relative_humidity_2m', 'pressure_msl', 'surface_pressure',
        'wind_speed_10m', 'cloud_cover', 'precipitation',
        'et0_fao_evapotranspiration', 'vapour_pressure_deficit'
    ]]

    final_df = final_df.rename(columns={
        'temperature_2m': 'temp_avg',
        'apparent_temperature': 'feels_like',
        'dew_point_2m': 'dew_point',
        'relative_humidity_2m': 'humidity',
        'pressure_msl': 'pressure',
        'surface_pressure': 'surface_pressure',
        'wind_speed_10m': 'wind_speed',
        'cloud_cover': 'cloud_cover',
        'precipitation': 'precipitation_mm',
        'et0_fao_evapotranspiration': 'evapotranspiration',
        'vapour_pressure_deficit': 'vpd'
    })
    excel_file = "indiancities_weather_q1_q4_openmeteo.xlsx"
    final_df.to_excel(excel_file, index=False, sheet_name="Weather Data")
    print(f"\n✓ Successfully saved weather data for all 50 cities to: {excel_file}")
    print(f"Total records: {len(final_df)}")
    print(f"Columns: {list(final_df.columns)}")
else:
    print("No data was retrieved.")


[1/50] Fetching for Mumbai
[2/50] Fetching for Delhi
[3/50] Fetching for Bangalore
[4/50] Fetching for Hyderabad
[5/50] Fetching for Ahmedabad
[6/50] Fetching for Chennai
[7/50] Fetching for Kolkata
[8/50] Fetching for Surat
[9/50] Fetching for Pune
[10/50] Fetching for Jaipur
[11/50] Fetching for Lucknow
[12/50] Fetching for Kanpur
[13/50] Fetching for Nagpur
[14/50] Fetching for Indore
[15/50] Fetching for Thane
[16/50] Fetching for Bhopal
[17/50] Fetching for Visakhapatnam
[18/50] Fetching for Pimpri-Chinchwad
[19/50] Fetching for Patna
[20/50] Fetching for Vadodara
[21/50] Fetching for Ghaziabad
[22/50] Fetching for Ludhiana
Rate limited for Ludhiana! Waiting 10s (retry 1/20)...
Rate limited for Ludhiana! Waiting 20s (retry 2/20)...
Rate limited for Ludhiana! Waiting 30s (retry 3/20)...
[23/50] Fetching for Agra
[24/50] Fetching for Nashik
[25/50] Fetching for Faridabad
[26/50] Fetching for Meerut
[27/50] Fetching for Rajkot
[28/50] Fetching for Kalyan-Dombivli
[29/50] Fetching for